In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install stable_baselines3
!pip install numpy --upgrade 

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0 (from versions: 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.3.0


In [8]:
from gym import Env
from gym.spaces import Discrete, Box
from floris.tools import FlorisInterface
import matplotlib.pyplot as plt
import numpy as np
import math
import gym
import stable_baselines3.common.env_checker
import random
from os import path
from gym import spaces

In [17]:
class wind_farm(gym.Env):
    
    def __init__(self):
        # Actions we can take are Absolute Angle Representation (theta(0-360 deg))
        self.action_space = Box(low=np.array([0,0]),high=np.array([360,360]))
        # dataframe least and highest values
        self.observation_space = Box(low=np.array([0,1,1,0,-1,-0.5]), high=np.array([360,18,18,1,1.5,0.5]))
        # Set state space
        n=2
        self.state = np.array([random.randint(0,n-1),random.sample(range(1,18),2),random.sample(range(0,360),2)])

    
    def farm_power(self,wind_d,wind_s,s1,s2):
        fi = FlorisInterface("inputs/gch.yaml")
        
        fi.floris.solver

        # 2. Modify the inputs with a more complex wind turbine layout
        D = 126.0  # Design the layout based on turbine diameter
        x = [6 * D,0]
        y = [5 * D, 0]

        # Pass the new data to FlorisInterface
        fi.reinitialize(
            layout_x=x,
            layout_y=y,
            wind_directions=wind_d,
            wind_speeds=wind_s
        )

        # 3. Calculate the velocities at each turbine for all atmospheric conditions
        # All turbines have 0 degrees yaw
        fi.calculate_wake()

        # 4. Get the total farm power
        turbine_powers = fi.get_turbine_powers() / 1000.0  # Given in W, so convert to kW
        farm_power_baseline = np.sum(turbine_powers, 2)  # Sum over the third dimension

        # 5. Develop the yaw control settings
        yaw_angles = np.zeros( (2, 1, 2) )  # Construct the yaw array with dimensions for two wind directions, one wind speed, and two turbines
        yaw_angles[0, :, 0] = s1         # At 270 degrees, yaw the first turbine 25 degrees
        yaw_angles[0, :, 1] = s2            # At 270 degrees, yaw the second turbine 25 degrees
        #yaw_angles[1, :, 0] = 10           # At 265 degrees, yaw the first turbine -25 degrees
        #yaw_angles[1, :, 1] = 10           # At 265 degrees, yaw the second turbine -25 degrees

        # 6. Calculate the velocities at each turbine for all atmospheric conditions with the new yaw settings
        fi.calculate_wake(yaw_angles=yaw_angles)

        # 7. Get the total farm power
        turbine_powers = fi.get_turbine_powers() / 1000.0
        farm_power_yaw = np.sum(turbine_powers, 2)

        # 8. Compare farm power with and without wake steering
        #difference = 100 * (farm_power_yaw - farm_power_baseline) / farm_power_baseline
        #print("Power % difference with yaw")
        #print(f"    270 degrees: {difference[0, 0]:4.2f}%")
        #print(f"    280 degrees: {difference[1, 0]:4.2f}%")
        return np.amax(farm_power_yaw)

    def step(self, action):
        global max_pow 
        max_pow=-1000
        speed = env.observation_space.sample()[1:3]
        # Apply action
        #Absolute angle representation formula
#       print(env.farm_power([270,265],[8],25,25))
        #print(action[1])
        self.state1 = action[0] - 0*math.radians(180)
        self.state2 = action[1] - 1*math.radians(180)
        # Reduce shower length by 1 second
        #self.shower_length -= 1 
#         a=[action[0],action[1]]
        # Calculate reward
#         print(type(float(speed)),"speed value",speed)
        
        if self.state1 >= 0 and self.state2 >= 0: 
            r = env.farm_power(action,speed,self.state1,self.state2)
            if r>max_pow:
                self.reward = env.farm_power(action,speed,self.state1,self.state2)
                max_pow=r
        #any angle negative,reward is negative
            else: 
                self.reward = -1 
        
        # Check if optimization is done
        if self.time_length <= 0: 
            done = True
        else:
            done = False
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.observation_space.sample(), self.reward, done, info

    def render(self):
        # Implement viz
        pass
    
    #Assuming that the agent is coming back to original state
    def reset(self):
        self.time_length = 60
        return self.observation_space.sample()

In [18]:
env = wind_farm()
stable_baselines3.common.env_checker.check_env(env , warn= True) 

C:\Users\manoh\AppData\Local\Temp\ipykernel_13132\2041970007.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.state = np.array([random.randint(0,n-1),random.sample(range(1,18),2),random.sample(range(0,360),2)])


In [19]:
from stable_baselines3.sac.policies import CnnPolicy, MlpPolicy, MultiInputPolicy
from stable_baselines3.sac.sac import SAC
from stable_baselines3 import A2C
import stable_baselines3.common.env_checker

__all__ = ["CnnPolicy", "MlpPolicy", "MultiInputPolicy", "SAC"]

In [31]:


model = SAC(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=5)
model.save("sac_wind_farm")

 #del model remove to demonstrate saving and loading

#model = SAC.load("sac_wind_farm")

obs = env.reset()

episodes = 10
#done = False
score = 0
rew = []
for episode in range(1, episodes+1):
    obs = env.reset() 
    #done = False
    #score = 0
    action, _states = model.predict(obs)
    #n_state, reward, done, info,_ = env.step(action,speed)
    obs, reward, done, info = env.step(action)
    #wind_direction - obs[0]
    #speed1 - obs[1]
    #speed2 - obs[2]
    #TI - obs[3]
    #vear - obs[4]
    #sheer - obs[5]
    print('Wind_Direction:  {}\n'.format(obs[0]))
    print('Speed1:  {}\n '.format(obs[1]))
    print('Speed2:  {}\n '.format(obs[2]))
    print('Turbulance Intensity:  {}\n '.format(obs[3]))
    print('Vear:  {}\n '.format(obs[4]))
    print('Sheer:  {}\n '.format(obs[5]))
    score=reward
    rew.append(score)
    #print(reward)
    '''
    while not done:
        #env.render()
        #action = env.action_space.sample()
        action, _states = model.predict(obs)
        #n_state, reward, done, info,_ = env.step(action,speed)
        obs, reward, done, info = env.step(action)
        if done=True:
            break
        else:
            score+=reward
    '''
        
    print('Episode:  {} Score:  {}'.format(episode, score))
    print("------------------------------------")
#while True:
    #action, _states = model.predict(obs)
    #obs, rewards, dones, info = env.step(action)

    #env.render()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wind_Direction:  342.8791809082031

Speed1:  10.769206047058105
 
Speed2:  7.752890110015869
 
Turbulance Intensity:  0.053053904324769974
 
Vear:  -0.9198594093322754
 
Sheer:  0.18761084973812103
 
Episode:  1 Score:  9999.971261469065
------------------------------------
Wind_Direction:  91.87516784667969

Speed1:  8.495718955993652
 
Speed2:  11.124476432800293
 
Turbulance Intensity:  0.6975870728492737
 
Vear:  0.6169078946113586
 
Sheer:  -0.42025521397590637
 
Episode:  2 Score:  7085.653259611353
------------------------------------
Wind_Direction:  87.5073013305664

Speed1:  7.4270501136779785
 
Speed2:  5.010993480682373
 
Turbulance Intensity:  0.9224675297737122
 
Vear:  0.047463126480579376
 
Sheer:  -0.23943054676055908
 
Episode:  3 Score:  10000.001058149019
------------------------------------
Wind_Direction:  52.30156707763672

Speed1:  8.194543838500977
 
Speed2:  11.008455

In [32]:
print("Mean of Power: ",statistics.mean(rew))

Mean of Power:  6750.082460203003


In [33]:
print("StdDev of Power",statistics.stdev(rew))

StdDev of Power 4651.6878650961025
